**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install parsley

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import parsley
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
plot_kwds = dict(node_color='w',
                 node_size=2000, linewidths=1,
                 edgecolors='k', layout='dot')

def add_node(G, AST, labels, parent=None):
    if AST is None:
        return
    
    nid = len(G.nodes)
    
    if parent is None:
        G.add_node(nid)
    else:
        G.add_node(nid)
        G.add_edge(parent, nid)

    if isinstance(AST, tuple):
        labels[nid] = AST[0]
        for child in AST[1:]:
            add_node(G, child, labels, nid)
    elif isinstance(AST, list):
        labels[nid] = 'LIST'
        for child in AST:
            add_node(G, child, labels, nid)
    else:
        labels[nid] = AST

def draw(AST):
    G = nx.DiGraph()
    labels = {}
    add_node(G, AST, labels)
    G.graph['graph'] = {'rankdir': 'TD'}
    pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')
    return nx.draw(G, pos=pos, labels=labels, **plot_kwds)

def ensure_list(e):
    if isinstance(e, list):
        return e
    else:
        return [e]

## Jazyky, gramatiky a parsery

V predchádzajúcom notebook-u sme sa venovali regulárnym výrazom. Regulárne výrazy opisujú regulárne jazyky, ktoré však predstavujú len jeden špeciálny typ formálnych jazykov. Formálnym jazykom $L$ všeobecne rozumieme **množinu reťazcov**  (viet). Hovoríme, že vety patria do jazyka $L$. Reťazce pozostávajú zo symbolov. Množinu týchto symbolov $\Sigma$ nazývame **abecedou jazyka**  $L$ [[intro_langs]](#intro_langs).

### Množinový opis jazyka

Keďže formálny jazyk je vlastne reprezentovaný množinou, prvým spôsobom, ako opísať formálny jazyk, je množinový zápis. Ak napríklad abeceda jazyka bude $\Sigma = \{ a, b \}$, potom vety sa budú skladať z rôznych kombinácií znakov $a$ a $b$ [[intro_langs]](#intro_langs). Jazyk by mohol potom vyzerať napríklad takto:

$$
L = \{ a, b, aa, bb \}.
$$
Prázdny reťazec "" sa v kontexte formálnych jazykov zvykne označovať $\lambda$ [[intro_langs]](#intro_langs). Ak by sme teda chceli, aby do jazyka patril aj  prázdny reťazec, mohli by sme písať:

$$
L = \{ \lambda, a, b, aa, bb \}.
$$
Pri množinových zápisoch jazykov sa tiež dá vyjadriť dovolený počet opakovaní znakov a pod., napr.:

$$
L = \{ a^n b^n | n \geq 0 \}
$$
by vyjadrovalo, že znaky $a$ a $b$ sa smú opakovať $n$ ráz, pričom $n \geq 0$ [[intro_langs]](#intro_langs).

### Opis jazyka cez gramatiky

Množinový opis formálnych jazykov v žiadnom prípadne nepredstavuje jedinú možnosť, ako ich opísať — spôsobov je viacero. Jedným veľmi silným spôsobom je opis pomocou gramatík. Gramatikou $G$ rozumieme $n$-ticu $\langle V, T, S, P \rangle$, pričom [[intro_langs]](#intro_langs):

* $V$ je konečná množina premenných;
* $T$ je konečná množina terminálnych symbolov;
* $S \in V$ je počiatočná premenná (start variable);
* $P$ je konečná množina **produkčných pravidiel** .
Vety patriace do jazyka $L$ opísaného gramatikou $G$ sú potom tie, ktoré vieme získať, ak začneme s počiatočnou premennou a budeme ľubovoľný počet ráz a v ľubovoľnom poradí aplikovať produkčné pravidlá – tak, aby sme získali postupnosť obsahujúcu len terminály.

Súvisiace koncepty bude najjednoduchšie ilustrovať na malom príklade. Dajme tomu, že máme gramatiku
$G = \langle \{ S \}, \{ a, b \}, S, P \rangle$, pričom $P$ obsahuje dve produkčné pravidlá [[intro_langs]](#intro_langs):

$$
\begin{aligned}
S &\rightarrow aSb \\
S &\rightarrow \lambda
\end{aligned}
$$
Pravidlá hovoria, že $S$ smieme nahradiť buď prázdnym reťazcom $\lambda$ alebo kombináciou $aSb$. Ak by sme teda začali od počiatočnej premennej $S$ a aplikovali druhé produkčné pravidlo, zistíme, že jazyk $L$ obsahuje prázdny reťazec $\lambda$. Ak by sme najprv aplikovali pravidlo 1 a potom pravidlo 2, získali by sme ďalšiu možnú vetu: $ab$. A tak by sme mohli pokračovať ďalej, takže ekvivalentný množinový zápis by bol v tomto prípade evidentne

$$
L = \{ a^n b^n | n \geq 0 \},
$$
t.j. jazyk, s ktorým sme sa už stretli vyššie.

#### Parsery

Ak vieme pre jazyk definovať gramatiku, existujú nástroje, ktoré vedia pre ňu automaticky vytvoriť parser. Parser vie overiť či postupnosť symbolov patrí do jazyka zodpovedajúceho danej gramatike a tiež vie na postupnosti vykonať iné potrebné operácie: napríklad ju transformovať do formy abstraktných syntaktických stromov, s ktorou pracujú kompilátory.

#### Lexery

V kontexte programovacích jazykov neoperujú väčšinou parsery priamo na surovom texte – väčšinou sa text najprv predspracuje lexerom, ktorý ho rozdelí na symboly (tokens) zodpovedajúce napr. numerickým literálom, indentifikátorom a pod.: lexery sú typicky založené na regulárnych výrazoch.

### Gramatika pre kalkulačku; Parsley

Ak sa má parsovať väčší objem textov, je vhodné použiť nástroje ako sú Lex a Yacc alebo Flex a Bison, ktoré vedia z definície symbolov a gramatiky automaticky vygenerovať rýchly lexer a parser v jazyku C alebo C++. Pre naše (najmä pedagogické) potreby však použijeme omnoho jednoduchší nástroj: Python-ový balíček `parsley`, ktorý kombinuje funkciu lexera aj parsera v jednoduchom Python-ovom rozhraní.

Ako príklad si zadefinujeme gramatiku pre jednoduchú kalkulačku.

#### Celé a reálne čísla

V prvom kroku si napíšeme gramatiku na parsovanie celých a reálnych čísel. Práve túto časť úlohy by tradične riešil lexer. Na vytvorenie gramatiky použijeme funkciu `parsley.makeGrammar`, ktorej dáme na vstup reťazec opisujúci jej pravidlá. Aby sme mohli pohodlne písať viacriadkový zápis, reťazec obalíme do troch párov úvodzoviek. V rámci balíčka `parsley` vieme definovať zhodu buď priamo s jednotlivými znakmi – v tom prípade ich v rámci pravidla obalíme do úvodzoviek – alebo vieme použiť niektoré z už preddefinovaných základných pravidiel [[parsley_ref]](#parsley_ref):

* **anything:**  zhoduje sa s ľubovoľným jedným vstupným znakom;
* **letter:**  zhoduje sa s jedným ASCII písmenom;
* **digit:**  zhoduje sa s ľubovoľnou desiatkovou číslicou;
* **letterOrDigit:**  kombinuje predchádzajúce 2 vzory;
* **end:**  zhoduje sa s koncom reťazca;
* **ws:**  zhoduje sa s nula alebo viacerými medzerami, tabulátormi a znakmi nového riadka.
Podobne ako v regulárnych výrazoch vieme tiež definovať opakovanie pomocou operátorov `+`, `*`, `{n}`, prípadne alternatívy pomocou operátora `|`. Ďalšiu notáciu, ktorá je k dispozícii, je možné nájsť v [[parsley_ref]](#parsley_ref).

My si teda ako prvé definujme pravidlo `int`, ktoré identifikuje celé číslo, t.j. postupnosť jednej alebo viacerých číslic a skúsme ho aplikovať na testovací reťazec:



In [ ]:
G = parsley.makeGrammar("""
int = digit+
""", {})

G("123").int()

Ako vidno, z pravidiel gramatiky generuje `parsley` Python-ové funkcie s rovnakým názvom, ktoré môžeme zavolať potom, ako dáme na vstup gramatiky nejaký reťazec. Pravidlá bežne fungujú tak, že navracajú zhodu v rámci posledného symbolu pravidla. V tomto prípade to je symbol `digit+`. Keďže ide o symbol s opakovaním, pravidlo nám vracia zoznam celej zhodujúcej sa postupnosti.

Keďže dosť často potrebujeme získať nejakú časť zhody naspäť v podobe textového reťazca, existuje na to v `parsley` špeciálna notácia: príslušnú časť pravidla stačí obaliť do špicatých zátvoriek:



In [ ]:
G = parsley.makeGrammar("""
int = <digit+>
""", {})

G("123").int()

Teraz sme namiesto zoznamu získali textový reťazec zodpovedajúci danému celému číslu. Lepšie by však bolo získať číslo rovno v numerickom tvare. Našťastie vieme do pravidiel zapísať aj inline Python-ový kód, ktorý umožňuje návratovú hodnotu ľubovoľne spracovať. Píše sa na koniec príslušnej časti pravidla a oddeľuje sa značením `->`. Aby sme v rámci kódu mohli pracovať so zhodami nájdenými v rámci pravidla, môžeme si ich pomenovať tak, že za ne napíšeme dvojboku a identifikátor. V našom prípade by teda mohli konvertovať reťazec na celé číslo pomocou funkcie `int` takto:



In [ ]:
G = parsley.makeGrammar("""
int = <digit+>:x -> int(x)
""", {})

G("123").int()

Na dovysvetlenie: časť `<digit+>:x` teraz priraďuje príslušnú zhodu do premennej `x` a časť `-> int(x)` hovorí, že má pravidlo navrátiť hodnotu `int(x)`.

Ďalej si skúsme definovať pravidlo pre reálne čísla: bude podobné ako pre celé čísla, ibaže v zápise bude práve jedna desatinná bodka:



In [ ]:
G = parsley.makeGrammar("""
float = <digit+ '.' digit+>:x -> float(x)
""", {})

G("3.14").float()

Vo všeobecnosti môže gramatika samozrejme obsahovať aj viacero pravidiel, ktoré sa jedno na druhé odvolávajú. Dajme tomu, že by v rámci gramatiky mali už vytvorené pravidlá `int` a `float` a pridali by sme ďalšie pravidlo `number` vyjadrujúce, že  číslo môže byť buď `int` alebo `float`:



In [ ]:
G = parsley.makeGrammar("""
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

print(G("123").number())
print(G("3.14").number())

#### Sčítanie

V ďalšom kroku môžeme pridať podporu sčítania. Pridáme ďalšie pravidlo, ktoré vyjadrí, že sa smú v jazyku vyskytovať aj viaceré čísla spojené symbolom `+` a že návratovou hodnotou bude v takom prípade súčet oboch čísel. Alternatívne dovolíme, aby sa zadalo aj jedno samotné číslo, preto bude nové pravidlo `addExpr` mať dve alternatívne definície, ktoré môžeme vyjadriť buď pomocou operátor `|` alebo zodpovedajúcimi definíciami `addExpr` v dvoch osobitných riadkoch:



In [ ]:
G = parsley.makeGrammar("""
addExpr = number:x1 '+' number:x2 -> x1 + x2
addExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

print(G("2+3").addExpr())
print(G("123").addExpr())

Táto gramatika nám zatial neumožňuje sčítať viac než dve čísla — taká veta nie je v rámci definovaného jazyka prípustná. Môžeme ju ale doplniť zavedením rekurzívneho pravidla takto:



In [ ]:
G = parsley.makeGrammar("""
addExpr = addExpr:x1 '+' addExpr:x2 -> x1 + x2
addExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

print(G("2+3+4").addExpr())

#### Odčítanie

Doplniť odčítanie bude jednoduché. Samotné výpočty kvôli zjednodušeniu zápisu vykonáme pomocou na to definovanej funkcie `calc`, ktorá bude mať na vstupe operátor a dve čísla:



In [ ]:
def calc(op, x1, x2):
    if op == '+':
        return x1 + x2
    elif op == '-':
        return x1 - x2
    elif op == '*':
        return x1 * x2
    elif op == '/':
        return x1 / x2
    else:
        raise RuntimeError("Uknown operator '{}'.".format(op))

Samotný operátor odčítania stačí potom v rámci pravidla jednoducho pridať ako ďalšiu alternatívu. Samozrejme budeme potrebovať vedieť, ktorý operátor sa použil, takže zhodu priradíme do premennej `op`.

Aby sme mohli v rámci gramatiky použiť vlastnú funkciu, treba ju tiež zaregistrovať pomocou slovníka, ktorý dávame na vstup funkcie `parsley.makeGrammar`. Povieme, že v gramatike funkcii s názvom `'calc'` zodpovedá Python-ová funkcia `calc`, t.j. `{'calc': calc}`.



In [ ]:
G = parsley.makeGrammar("""
addExpr = addExpr:x1 ('+'|'-'):op addExpr:x2 -> calc(op, x1, x2)
addExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {'calc': calc})

print(G("2+3-4").addExpr())

#### Násobenie a delenie

Podstatne zložitejšie bude doplniť násobenie a delenie: pretože tie majú podľa konvencie prioritu oproti sčítaniu a odčítaniu. Vyriešime to pridaním nového typu výrazu (pravidla) `multExpr`, pričom dáme pozor, aby sme pravidlá kombinovali takým spôsobom a definovali v takom poradí, aby sa všetko násobenie a delenie muselo zrealizovať ešte predtým než sa budú dať dané výrazy dosadiť do pravidla `addExpr`.



In [ ]:
G = parsley.makeGrammar("""
expr = addExpr
addExpr = addExpr:x1 ('+' | '-'):op multExpr:x2 -> calc(op, x1, x2)
addExpr = multExpr
multExpr = multExpr:x1 ('*' | '/'):op multExpr:x2 -> calc(op, x1, x2)
multExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {'calc': calc})

print(G("2*3+4").expr())
print(G("4+2*3").expr())

Ak táto gramatika narazí najprv na operátor `+` (alebo `-`; pravidlo `addExpr`), nemôže sa aplikovať hneď, pretože pravidlo je definované tak, že jeden alebo oba sčítance musia vyhovovať `multExpr`: musí sa teda najprv overiť či to platí a v rámci toho kroku sa vykonajú príslušné násobenia.

#### Doplnenie zátvoriek

Ak chceme prioritu operácií explicitne zmeniť, dá sa to urobiť pomocou zátvoriek. Doplniť by sme ich mohli ako ďalšiu možnú alternatívu pravidla `multExpr`:



In [ ]:
G = parsley.makeGrammar("""
expr = addExpr
addExpr = addExpr:x1 ('+' | '-'):op multExpr:x2 -> calc(op, x1, x2)
addExpr = multExpr
multExpr = multExpr:x1 ('*' | '/'):op multExpr:x2 -> calc(op, x1, x2)
multExpr = number
multExpr = '(' addExpr:x ')' -> x
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {'calc': calc})

print(G("2*(3+4)").expr())
print(G("(3+4)*2").expr())

### Abstraktné syntaktické stromy

V praxi sa často stretávame s tým, že nad výsledkami parsovania chceme vykonať zložitejšie analýzy: v kontexte kompilátorov sa napríklad realizuje pred prekladom do strojového kódu optimalizácia. Zo zdrojového kódu sa teda pomocou parsera vytvorí tzv. abstraktný syntaktický strom, ktorý sa ľahšie analyzuje.

Na ilustráciu teraz skúsime prepísať gramatiku vyššie vytvorenej kalkulačky tak, aby namiesto priamych výpočtov vytvorila abstraktný syntaktický strom, ktorý si následne vizualizujeme. Urobíme to tak, že namiesto operácie `calc` budeme vytvárať trojice `(op, x1, x2)` obsahujúce operátor `op` a ľavý a pravý výraz, na ktoré sa operátor aplikuje. Vznikne nám tak jednoduchý strom zložený z vnorených trojíc.



In [ ]:
G = parsley.makeGrammar("""
expr = addExpr
addExpr = ( addExpr:x1 ('+' | '-'):op multExpr:x2 -> (op, x1, x2)
          | multExpr)
multExpr = ( multExpr:x1 ('*' | '/'):op number:x2 -> (op, x1, x2)
           | number | '(' expr:x ')' -> x)
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

AST = G("(2*2+3*2)*4").expr()
print(AST)
draw(AST)

### Parsovanie HTML

V notebook-u o regulárnych výrazoch sme ukázali, ako sa dajú z textu odstrániť HTML tagy, povedali sme však, že na to, aby sme odstránili HTML tag-y aj s ich obsahom nie sú regulárne výrazy dostatočne expresívne: keďže nevedia vyjadriť rekurzívne vzory, nepodarilo by sa nám pomocou nich sledovať otváranie a uzatváranie tagov.

V ďalšom príklade si definujeme gramatiku, ktorá umožní parsovať HTML a zobrazíme výsledný abstraktný syntaktický strom. Pre zjednodušenie budeme predpokladať, že všetky použité tagy budú párové (bolo by možné samozrejme pracovať aj s nepárovými tagmi, ale pre naše potreby by to úlohu zbytočne skomplikovalo).

#### Text okrem znakov `<` a `>`

Ako prvé si vytvorme pravidlo na zhodu s ľubovoľným textom okrem znakov `<` a `>` označujúcich tagy. Použijeme na to špeciálny zápis `?(condition)`, ktorý nám umožňuje zhodu vyhodnotiť pomocou inline Python-ového kódu.



In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
text = <except_angle+>
""", {})

print(G("abcd").text())

#### Otvárajúci a uzatvárajúci tag

Ďalej si vytvorme pravidlá pre otvárajúci a uzatvárajúci tag:



In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
""", {})

print(G("<tag>").opening_tag())
print(G("</tag>").closing_tag())

#### Páry tagov

Ďalej zadefinujme páry tagov. Pár musí obsahovať otvárajúci a uzatvárajúci tag a oba tagy sa musia zhodovať. Medzi tagmi môže byť text alebo ďalší tag.



In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
expr = opening_tag:t1 expr:e closing_tag:t2 ?(t1 == t2) -> ('<' + t1 + '>', e)
expr = <except_angle+>
""", {})

print(G("<b>content</b>").expr())
print(G("<b><div>content</div></b>").expr())

#### Reťazenie výrazov

Tagy môžu byť obklopené aj textom, ďalšími tagmi a rovnako môžu kombináciu viacerých tagov aj textov aj obsahovať. Vytvoríme preto pravidlo `expr = expr:e1 expr:e2`, ktoré umožní výrazy reťaziť. Použijeme pomocnú funkciu `ensure_list`, ktorá výrazy v prípade potreby obalí do zoznamov a spojí ich dokopy: v dôsledku toho budú všetky výrazy (tagy aj text) na tej istej úrovni spojené do jedného zoznamu.



In [ ]:
text = """
text above
<div>
div 1 content
<span>inner span 1</span>
<span>inner span 2</span>
</div><div>
div 2 content
<span>inner span</span>
</div>
text below
"""

In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
expr = opening_tag:t1 expr:e closing_tag:t2 ?(t1 == t2) -> ('<' + t1 + '>', e)
expr = expr:e1 expr:e2 -> ensure_list(e1) + ensure_list(e2)
expr = <except_angle+> -> "txt"
""", {'ensure_list': ensure_list})

AST = G(text).expr()
plt.figure(figsize=(10, 5))
draw(AST)

#### Odstránenie HTML tagov a ich obsahu

V ďalšom kroku ešte prepracujme gramatiku tak, aby sme splnili úlohu z predchádzajúceho notebook-u. Odstránime všetky HTML tagy aj s ich obsahom.



In [ ]:
text = """
text above
<div>
div 1 content
<span>inner span 1</span>
<span>inner span 2</span>
</div><div>
div 2 content
<span>inner span</span>
</div>
text below
"""

In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
expr = opening_tag:t1 expr:e closing_tag:t2 ?(t1 == t2) -> ('')
expr = expr:e1 expr:e2 -> e1 + e2
expr = <except_angle+>
""", {})

print(G(text).expr())

---
### Úloha: parsovanie zoznamu

**Nasledujúca bunka definuje premennú `text`, ktorá obsahuje textovú reprezentáciu zoznamu. Napíšte gramatiku `G` s hlavným pravidlom `list`, ktorá `text` sparsuje späť do podoby zoznamu reťazcov.** 

---


In [ ]:
lst = ["abcd", "efg", "hij", "klmn"]
text = ",".join(lst)
print(text)

In [ ]:
G = parsley.makeGrammar("""


# ---


""", {})

#### Testovanie

A teraz skontrolujme, či sa list sparsoval korektne.



In [ ]:
l = G(text).list()

if l == lst:
    print("List parsed correctly.")
else:
    print("List parsed correctly.")
    print("-- Expected: {}".format(l))
    print("-- Got: {}".format(lst))

### References

<a id="intro_langs">[intro_langs]</a> Linz, P., 2006. An introduction to formal languages and automata. Jones & Bartlett Learning.

<a id="parsley_ref">[parsley_ref]</a> Parsley Reference. URL: <https://parsley.readthedocs.io/en/latest/reference.html>

